In [2]:
import undetected_chromedriver as uc
# from selenium import webdriver
from time import sleep as time_sleep
from bs4 import BeautifulSoup as BS
import requests
import subprocess
from config import headers, cookies
from pprint import pprint
import lxml
import pandas as pd

In [6]:
url_list = '''
https://novelbin.com/b/genetic-ascension
'''.strip().split('\n')


url = url_list[0]

host = '/'.join(url.split('/', maxsplit=3)[:3])
name_title = url.split('/')[-1]
print(name_title)

chapters_link = f'{host}/ajax/chapter-option?novelId={name_title}&currentChapterId'
print(chapters_link)

genetic-ascension
https://novelbin.com/ajax/chapter-option?novelId=genetic-ascension&currentChapterId


In [4]:
cookies = {
    'cf_clearance': 'KvPRwRyGr95Yo7CFRDYwHSGPwCYC9b5Qc63YSEWb1NI-1718403960-1.0.1.1-XwC363irbNHOsE1QOL0fvN8TEUUb_a1EY4HKGZTXRdNOaPTR4qwzVc5hAapVNSMDg_xQ.TXmlAaHwaRdR9zNPg',
}

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,de;q=0.6',
    # 'cache-control': 'max-age=0',
    # 'priority': 'u=0, i',
    # 'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    # 'sec-ch-ua-mobile': '?0',
    # 'sec-ch-ua-platform': '"Windows"',
    # 'sec-fetch-dest': 'document',
    # 'sec-fetch-mode': 'navigate',
    # 'sec-fetch-site': 'none',
    # 'sec-fetch-user': '?1',
    # 'upgrade-insecure-requests': '1',
    # 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
}

params = {
    'novelId':'lord-of-mysteries-2-circle-of-inevitability',
    'currentChapterId':'1'
}

tt = requests.get(
    'https://novelbin.com/ajax/chapter-option',
    cookies=cookies,
    headers=headers,
    params=params
)
tt.text

'<!DOCTYPE html>\n<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->\n<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->\n<head>\n<title>Attention Required! | Cloudflare</title>\n<meta charset="UTF-8" />\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=Edge" />\n<meta name="robots" content="noindex, nofollow" />\n<meta name="viewport" content="width=device-width,initial-scale=1" />\n<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/cf.errors.css" />\n<!--[if lt IE 9]><link rel="stylesheet" id=\'cf_styles-ie-css\' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->\n<style>body{margin:0;padding:0}</style>\n\n\n<!--[if gte IE 10]><!-->\n<script>\n  if (!navigator.cookieEnabled) {\n    window.addEventListene

In [7]:
list_links = []

# tt = requests.get(chapters_link, headers=headers, cookies=cookies)
links_value = BS(tt, 'html.parser')
tb_list = links_value.find_all('option')
elem : BS
# links_value
for elem in tb_list:
    list_links.append(elem.get('value'))
list_links
print('Всего ссылок',len(list_links))
print('Последняя ссылка:')
print(list_links[-1])

Всего ссылок 816
Последняя ссылка:
https://readnovel.noveljar.org/novel-book/lord-of-mysteries-2-circle-of-inevitability/chapter-816-dark-forest


In [55]:
driver = uc.Chrome()
# driver = webdriver.Chrome()

driver.get(host)
time_sleep(1)
driver.get(url)
for _ in range(200):
    page = driver.page_source
    if not 'cloudflare.com' in page:
        break
    else:
        time_sleep(1)
else:
    driver.quit()
print('yes')

HTTPError: HTTP Error 404: Not Found

In [5]:
book_eng = list()

for link in list_links[400:1000]:
    driver.get(link)
    for _ in range(200):
        page = driver.page_source
        if not 'cloudflare.com' in page:
            break
        else:
            time_sleep(1)
    else:
        driver.quit()
    time_sleep(0.5)
    soup = BS(page, 'html.parser')
    novel_title = soup.find(attrs={'class':'novel-title'})
    content = soup.find(attrs={'id':'chr-content'})
    
    contents = content.find_all('p')
    
    content_list = list()
    el: BS
    for el in contents:
        content_list.append(el.get_text())
    text = '\n'.join(content_list)
    book_eng.append(text)
len(book_eng)

600

In [6]:
driver.quit()

In [9]:
book_eng_full = '\n'.join(book_eng)
a = pd.Series(book_eng_full.split('\n')).str.strip()
b = a.value_counts(dropna=False).to_frame().query('count > 8')
b

,count
,35363
This material is derived from n0v𝓮lb𝕚n¶,84
The source of this material is novelbin★,80
You'll find the origin of this content at n0velb!n•,69
Trace the roots of this material to novelb!n•,67
The original content can be pinpointed to n0v3lbin•,67
This material is derived from n0v£lbin•,66
Explore the genesis of this content at n𝒐v@lbin,65
The source of this content can be linked to n0v𝓮lb𝕚n•,64
This content originates from n0v3lbin•,28


In [10]:
c = '\n'.join(a.drop(index=a[a.isin(b.index)].index, errors='ignore').values)

In [8]:
# # clean text:
# test = book_eng_full.split('\n')
# test2 = list()
# for row in test:
#     if 'The source of this content is' in row:
#         continue
#     test2.append(row)
# book_eng_full = '\n'.join(test2)

# book_eng_full = book_eng_full.replace('6054d257f56b520818c0fb96','')
# book_eng_full = book_eng_full.replace('                  Ads by PubFuture','')
# book_eng_full = book_eng_full.replace('\n\n','\n').replace('\n\n','\n').replace('\n\n','\n')

In [11]:
with open(
    file=f'{name_title}.txt',
    mode='w',
    encoding='utf-8'
) as file:
    # file.write(book_eng_full)
    file.write(c)

In [12]:
with open(
    file=f'{name_title}.txt',
    encoding='utf-8'
) as file:
    book_eng_full = file.read()
# book_eng_full

In [13]:
from yandexfreetranslate import YandexFreeTranslate, YandexFreeTranslateError
from multiprocessing.dummy import Pool as ThreadPool
from tqdm.notebook import tqdm
from urllib.error import URLError

def _worker(que: tuple) -> str:
    for _ in range(10):
        queue, pbar = que
        if not queue:
            pbar.update(1)
            return ''
        yt = YandexFreeTranslate(api = "ios")
        try:
            text = yt.translate("en", "ru", queue)
        except TimeoutError:
            continue
        except URLError:
            time_sleep(1)
            continue
            
        except YandexFreeTranslateError:
            time_sleep(1)
            continue
            
        # text = text.replace('/n', '')
        pbar.update(1)
        return text
    raise Exception()

def run_translate(lst: list[str])-> list[str]:
    with tqdm(total=len(lst) ) as pbar:
        l_b = list(zip(lst, ( pbar for _ in range(len(lst)))))
        with ThreadPool(30) as pool:
            workreturn = pool.map(_worker, l_b)
    return workreturn

lst = book_eng_full.split('\n')

text_rus = run_translate(lst)

text_rus = '\n'.join(text_rus)

with open(
    file=f'{name_title}_rus.txt',
    mode='w',
    encoding='utf-8'
) as file:
    file.write(text_rus)

  0%|          | 0/36555 [00:00<?, ?it/s]